# Catalog Design Pattern

A Class that holds a dict of static methods that can be triggered based on a key/param



In [2]:
class Catalog:
    def __init__(self, param):
        self._static_method_choices = {
            'param_value_1': self._static_method_1,
            'param_value_2': self._static_method_2,
            'param_value_3': self._static_method_3,            
        }
    
        if param in self._static_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    @staticmethod
    def _static_method_1():
        print("Executed Method 1!")
    
    @staticmethod
    def _static_method_2():
        print("Executed Method 2!")
        
    @staticmethod
    def _static_method_3():
        print("Executed Method 3!")
        
    def main_method(self):
        self._static_method_choices[self.param]()
        
        
class CatalogInstance:
    def __init__(self, param):
        self.x1 = 'x1'
        self.x2 = 'x2'
        
        if param in self._instance_method_choices:
            self.param = param
            
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    def _instance_method_1(self):
        print(f"Value {self.x1}")
    
    def _instance_method_2(self):
        print(f"Value {self.x2}")
    
    _instance_method_choices = {
        'param_value_1': _instance_method_1,
        'param_value_2': _instance_method_2
    }
    
    def main(self):
        self._instance_method_choices[self.param].__get__(self)()
        
        
class CatalogClass:
    x1 = 'x1'
    x2 = 'x2'
    
    def __init__(self, param):
        if param in self._class_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid Value for Param: {param}")
    
    @classmethod
    def _class_method_1(cls):
        print(f"Value {cls.x1}")
    
    @classmethod
    def _class_method_2(cls):
        print(f"Value {cls.x2}")
    
    _class_method_choices = {
        'param_value_1': _class_method_1,
        'param_value_2': _class_method_2,
    }
    
    def main(self):
        self._class_method_choices[self.param].__get__(None, self.__class__)()
        

class CatalogStatic:
    def __init__(self, param):
        if param in self._static_method_choices.keys():
            self.param = param
        else:
            raise ValueError(f"Invalid value for Param: {param}")
        
    @staticmethod
    def _static_method_1():
        print("executed method 1!")

    @staticmethod
    def _static_method_2():
        print("executed method 2!")

    _static_method_choices = {'param_value_1': _static_method_1, 'param_value_2': _static_method_2}

    def main(self):
        self._static_method_choices[self.param].__get__(None, self.__class__)()
        

def main():
    test0 = Catalog('param_value_2')
    test0.main_method()
    
    test1 = Catalog('param_value_1')
    test1.main_method()
    
    test2 = CatalogInstance('param_value_1')
    test2.main()
    
    test3 = CatalogInstance('param_value_2')
    test3.main()
    
    test4 = CatalogStatic('param_value_1')
    test4.main()
    
    test5 = CatalogStatic('param_value_2')
main()

Executed Method 2!
Executed Method 1!
Value x1
Value x2
executed method 1!


# Chain of Responsibility

In [15]:
import abc


class Handler(metaclass=abc.ABCMeta):
    def __init__(self, successor=None):
        self.successor = successor
        
    def handle(self, request):
        res = self.check_range(request)
        if not res and self.successor:
            self.successor.handle(request)
    
    @abc.abstractmethod
    def check_range(self, request):
        pass
    

class ConcreteHandler0(Handler):
    @staticmethod
    def check_range(request):
        if 0 <= request < 10:
            print(f"request {request} handled in handler 0")
            return True

        
class ConcreteHandler1(Handler):
    """... With it's own internal state"""

    start, end = 10, 20

    def check_range(self, request):
        if self.start <= request < self.end:
            print(f"request {request} handled in handler 1")
            return True

class ConcreteHandler2(Handler):
    def check_range(self, request):
        start, end = self.get_interval_from_db()
        if start <= request < end:
            print(f"request {request} handled in handler 2")
            return True
    
    @staticmethod
    def get_interval_from_db():
        return (20, 30)
    

class FallbackHandler(Handler):
    @staticmethod
    def check_range(request):
        print(f"End of Chain, No Handler for {request!r}")
        return False

def main():
    """
    >>> h0 = ConcreteHandler0()
    >>> h1 = ConcreteHandler1()
    >>> h2 = ConcreteHandler2(FallbackHandler())
    >>> h0.successor = h1
    >>> h1.successor = h2
    >>> requests = [2, 5, 14, 22, 18, 3, 35, 27, 20]
    >>> for request in requests:
    ...     h0.handle(request)
    request 2 handled in handler 0
    request 5 handled in handler 0
    request 14 handled in handler 1
    request 22 handled in handler 2
    request 18 handled in handler 1
    request 3 handled in handler 0
    End of Chain, No Handler for 35
    request 27 handled in handler 2
    request 20 handled in handler 2
    """

import doctest
doctest.testmod(optionflags=doctest.ELLIPSIS)

TestResults(failed=0, attempted=7)

# Chaining Method

In [18]:
class Person:
    def __init__(self, name, action):
        self.name = name
        self.action = action
        
    
    def do_action(self):
        print(self.name, self.action.name, end=' ')
        return self.action
    

class Action:
    def __init__(self, name):
        self.name = name
        
    def amount(self, val):
        print(val, end=' ')
        return self
    
    def stop(self):
        print('then stop')


def main():
    """
    >>> move = Action('move')
    >>> person = Person('Jack', move)
    >>> person.do_action().amount('5m').stop()
    Jack move 5m then stop
    """

import doctest
doctest.testmod()

TestResults(failed=0, attempted=3)

# Iterator Pattern

In [27]:
import functools


def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("--"*10)
        print(f"Calling {func!r} with {args!r}, {kwargs!r}")
        value = func(*args, **kwargs)
        print(f"Returned {func!r} with {value!r}")
        print("--"*10)
        return value
    return wrapper

@debug
def count_to(count):
    for number in range(0, count):
        yield number

# Test the generator
count_to_two = lambda: count_to(2)
count_to_five = lambda: count_to(5)

for number in count_to_two():
    print(number)
    
for number in count_to_five():
    print(number)

--------------------
Calling <function count_to at 0x112502d30> with (2,), {}
Returned <function count_to at 0x112502d30> with <generator object count_to at 0x112583890>
--------------------
0
1
--------------------
Calling <function count_to at 0x112502d30> with (5,), {}
Returned <function count_to at 0x112502d30> with <generator object count_to at 0x112583d60>
--------------------
0
1
2
3
4


# Iterator Pattern using Python Native Structure

In [31]:
class NumberWords:
    
    _WORD_MAP = ('one', 'two', 'three', 'four', 'five', 'six')
    
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.start > self.stop or self.start > len(self._WORD_MAP):
            raise StopIteration
        
        current = self.start
        self.start += 1
        return self._WORD_MAP[current-1]

for number in NumberWords(start=1, stop=5):
    print(number)

print('-'*50)
    
for number in NumberWords(start=1, stop=50):
    print(number)
    

one
two
three
four
five
--------------------------------------------------
one
two
three
four
five
six


# Mediator

Communicating with a mediator / broker. reduces coupling and complexity only handles by mediator

In [34]:
class User:
    def __init__(self, name):
        self.name = name
        self.chat_room = ChatRoom()
    
    def say(self, message):
        self.chat_room.display_message(self, message)
    
    def __str__(self):
        return self.name
    
class ChatRoom:
    def display_message(self, user, message):
        print(f"[{user} says]: {message}")

alamin = User('alamin')
anika = User('anika')
anamika = User('anamika')

alamin.say('Hi')
anika.say('Hi Back!')
anamika.say('Howdy! :)')


[alamin says]: Hi
[anika says]: Hi Back!
[anamika says]: Howdy! :)


# Memento

Provides the ability to restore an object to its previous state

In [19]:
from copy import deepcopy, copy


class NumObj:
    def __init__(self, value):
        self.value = value
        
    def increment(self):
        self.value += 1

    def __repr__(self):
        return f"{self.__class__.__name__!r} --> {self.value}"

        
def memento(obj, deep=False):
    state = deepcopy(obj.__dict__) if deep else copy(obj.__dict__)

    def rollback():
        obj.__dict__.clear()
        obj.__dict__.update(state)
    
    return rollback


class Transaction:
    states = []
    deep = False

    def __init__(self, deep, *targets):
        self.deep = deep
        self.targets = targets
        self.commit()
        
    def commit(self):
        self.states = [memento(target, self.deep) for target in self.targets]
        print(self.states)
        
    def rollback(self):
        for state in self.states:
            state()


n = NumObj(0)
print(n)

t = Transaction(False, n)

try:
    n.increment()
    n.increment()
    n.increment()
    print(n)

    t.commit()

    n.increment()
    n.increment()
    n.increment()
    print(n)

    n.value += 'x'
except Exception:
    t.rollback()
    print(n)

'NumObj' --> 0
[<function memento.<locals>.rollback at 0x10726d430>]
'NumObj' --> 3
[<function memento.<locals>.rollback at 0x10726d940>]
'NumObj' --> 6
'NumObj' --> 3
